In [24]:
import tkinter as tk 
from tkinter import  Message, Text
import os
import shutil
import csv
import numpy as np 
import cv2
from PIL import  Image, ImageTk
import pandas as pd
import datetime
import time
import tkinter.ttk as ttk
import tkinter.font as font
from pathlib import Path

## Set window properties

In [25]:
# constants
font_size = ('times', 15, 'bold')
green = 'green'
white = 'white'


In [26]:
window = tk.Tk()
window.title("Face Recognizer")
window.configure(background='white')
window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)
message = tk.Label(
   window, text="Face-Recognition-System",
   bg='green', fg='white', width=50, height=3, font=('times', 30, 'bold')  
)
message.place(x=200, y=20)

lbl = tk.Label(window, text="No.", width=20, height=2, fg=green, bg=white, font=font_size)
lbl.place(x=400, y=200)

txt = tk.Entry(window, width=20, fg=green, bg=white ,font=font_size)
txt.place(x=700, y=215)

lbl2 = tk.Label(window, text="Name", width=20, fg=green, bg=white, height=2, font=font_size)
lbl2.place(x=400, y=300)

txt2 = tk.Entry(window, width=20, bg=white, fg=green, font=font_size)
txt2.place(x=700, y=315)

function to Check whether a text is number or not

In [27]:
def isNumber(s):
   try:
      float(s)
      return True
   except ValueError:
      pass
   try:
      import unicodedata
      unicodedata.numeric(s)
      return True
   except (TypeError, ValueError):
      pass
   return False

function to take images for training

In [28]:
def takeImages():
   # Both ID and Name is used for recognizing the Image
   Id = (txt.get())
   name = (txt2.get())

   #check if the Id is numeric and name is alphabetical
   if(isNumber(Id) and name.isalpha()):
      # opening the primary camera if you want to access
      # the secondary camera you can mention the number as 1 inside the parenthesis
      cam = cv2.VideoCapture(1)

      # Specifying the path to haarcascade file
      harcascadePath = 'data\haarcascade_frontalface_default.xml'

      # Creating the classier based on the haarcascade file.
      detector = cv2.CascadeClassifier(harcascadePath)

      # initializing the sample number (No. of images) as 0
      sampleNum = 0 
      while(True):
         # Reading the video captures by camera frame by frame
         ret, img = cam.read()

         # converting the image into grayscale as most of 
         # the processing is done in grayscale format
         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

         # it converts the images in different sizes (decreases by 1.3 times)
         # and 5 specifies the number of times scaling happens
         faces = detector.detectMultiScale(gray, 1.3, 5)

         #for creating a rectangle around the image
         for (x,y,w,h) in faces:
            # specify the cordinates of the image as well as color and thickness of the rectangles,
            # incrementing sample number for each image
            cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0,0), 2)
            sampleNum += 1

            # save the captured face in the dataset folder TrainingImage as the image needs to be 
            # trained are saved in this folder
            cv2.imwrite("TrainingImage\ " + name +"." + Id + "." + str(sampleNum) + ".jpg", gray[y:y+h, x:x+w])

            # display the frame that has been captured and drawn rectangle around it
            cv2.imshow('frame', img)
         
         # wait for 100 milliseconds
         if cv2.waitKey(100) & 0xFF == ord('q'):
            break
         # break if the sample number is more than 60
         elif sampleNum > 60:
            break
      # releasing the resources
      cam.release()

      # closing all the windows
      cv2.destroyAllWindows()

      # displaying message for the user
      res = "Images Saved for ID : " + Id + "Name : " + name

      # creating the entry for the user in a csv file
      row = [Id, name]
      with open('UserDetails\ UserDetails.csv', 'a+') as csvfile:
         writer = csv.writer(csvfile)
         # Entry of the row in csv file
         writer.writerow(row)
      message.configure(text=res)
   elif (isNumber(Id)):
      res = "Enter Alphabetical Name"
      message.configure(text=res)
   elif (name.isalpha()):
      res = "Enter Numeric Id"
      message.configure(text=res)
      

Training the images saved in training image folder

In [29]:
def getImagesAndLabels(path):
      # get the path of all the files in the folder
      imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
      faces = []

      # create empyty Id list
      ids = []

      # now loop through all the image paths and load the Ids and the images saved in the folder
      for imagePath in imagePaths:
         # load the image and convert it to grayscale
         pillimage = Image.open(imagePath).convert('L')

         # now convert the PIL image into numpy array
         imageNp = np.array(pillimage, 'uint8')

         # get the Id from the image
         Id = int(os.path.split(imagePath)[-1].split(".")[1])

         # extract the face from the training image samples
         faces.append(imageNp)
         ids.append(Id)
         return faces, ids
      
def trainImages():
   # local Binary Pattern Histogram is a Face Recognizer algorithm inside OpenCV
   # module used for training the image dataset
   recognizer = cv2.face.LBPHFaceRecognizer_create()

   # specify the path for HaarCascade file
   harcascadePath = "data\haarcascade_frontalface_default.xml"

   # create detector for faces
   detector = cv2.CascadeClassifier(harcascadePath)

   # saving the detected faces in variables
   faces, Id = getImagesAndLabels("TrainingImage")

   # save the trained faces and their respective Id's in a model named as "trainner.yml"
   recognizer.train(faces, np.array(Id))
   recognizer.save("TrainingImageLabel\Trainner.yml")

   # displaying the message
   res = "Image Trained"
   message.configure(text=res)

for testing phase

In [30]:
def trackImages():
   recognizer = cv2.face.LBPHFaceRecognizer_create()

   # read the trained model
   recognizer.read("TrainingImageLabel\Trainer.yml")
   harcascadePath = "data\haarcascade_frontalface_default.xml"

   faceCascade = cv2.CascadeClassifier(harcascadePath)

   # get the name from 'userdetails.csv'
   df = pd.read_csv("UserDetails\ UserDetails.csv")
   cam = cv2.VideoCapture(0)
   font = cv2.FONT_HERSHEY_SIMPLEX  
   while True:
      ret, im = cam.read()
      gray = cv2.cvtColot(im, cv2.COLOR_BGR2GRAY)
      faces = faceCascade.detectMultiScale(gray, 1.2, 5)
      for (x,y,w,h) in faces:
         cv2.rectangle(im, (x,y), (x+w, y+h), (225, 0,0), 2)
         Id, conf = recognizer.predict(gray[y:y+h, x:x+w])
         if(conf < 50):
            aa = df.loc[df['Id'] == Id]['Name'].values 
            tt = str(Id) + "." + aa 
         else:
            Id = 'Unknown'
            tt = str(Id)
            if(conf > 75):
               noOfFile = len(os.listdir("imagesUnknown")) + 1
               cv2.imwrite("ImagesUnknown\Image" + str(noOfFile) + '.jpg', im[y:y+h, x:x+w])
               cv2.putText(im, str(tt), (x,y+h), font, 1, (255,255,255), 2)
               cv2.imshow('im', im)
      if(cv2.waitKey(1) == ord('q')):
         break
      cam.realease()
      cv2.destroyAllWindows()

In [31]:
takeImgBtn = tk.Button(window, text = "Sample", command=takeImages, fg=white, bg=green, width=20, height=3, activebackground="Red", font=font_size)
takeImgBtn.place(x=200, y=500)

trainImgBtn = tk.Button(window, text="Training", command=trainImages, fg=white, bg=green, width=20, height=3, activebackground="Red", font=font_size)
trainImgBtn.place(x=500, y=500)

trackImgBtn = tk.Button(window, text="Testing", command=trackImages, fg=white, bg=green, width=20, height=3, activebackground="Red", font=font_size)
trackImgBtn.place(x=800, y=500)

quitWindowBtn = tk.Button(window, text='Quit', command=window.destroy, fg=white, bg=green, width=20, height=3, activebackground="Red", font=font_size)
quitWindowBtn.place(x=1100, y=500)

In [32]:
window.mainloop()

[ WARN:0@363.856] global /croot/opencv-suite_1691620365762/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@363.857] global /croot/opencv-suite_1691620365762/work/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@363.857] global /croot/opencv-suite_1691620365762/work/modules/core/src/persistence.cpp (505) open Can't open file: 'data\haarcascade_frontalface_default.xml' in read mode
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kratosgado/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_37111/420980957.py", line 26, in takeImages
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.6.0) /croot/opencv-suite_1691620365762/work/modules/imgproc/s

In [ ]:
# bfdbfd